In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
print(torch.cuda.get_device_name(0))

In [3]:
#load data
import glob

#train_images = []
#test_images = []
train_images = glob.glob('../input/fingers/train/*.png')
test_images = glob.glob('../input/fingers/test/*.png')
myfingers = glob.glob('../input/myfingers/my_finger/*.jpg')

#print(len(myfingers))
print(len(train_images), len(test_images), len(myfingers))

In [4]:
# 기존 data에 myfinger 넣어주기 (뒤에 이 데이터들이 쏠려있겠지만, 데이터로드시 shuffle해주니 괜찮을 것이라고 생각.)
train_images.extend(myfingers[:20000])
test_images.extend(myfingers[20000:])
print(len(train_images), len(test_images))

In [5]:
import matplotlib.pyplot as plt
import cv2

plt.figure(figsize=(10, 10))
for i in range(16):
    ax = plt.subplot(4, 4, i + 1)
    if i <= 8:
        plt.imshow(cv2.imread(train_images[i]))
    else :
        plt.imshow(plt.imread(train_images[25500 - i]))
    plt.axis("off")

In [6]:
img = cv2.imread(train_images[20000])
height, width, channels = img.shape
print(height,width,channels)

In [7]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision.io import read_image

device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
print(device)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((256, 256))
])

In [8]:
train_labels = [int(x[-6:-5]) for x in train_images]

In [ ]:
a = np.array(train_labels) 
np.unique(a) # 라벨링이 잘되어있나 확인

In [ ]:
#커스텀 데이터셋 설정
#dataset and dataloader for train
class train_dataset(Dataset) :
    def __init__(self, image_list, image_labels, transform, device) :
        self.image_list = image_list
        self.image_labels = image_labels
        self.transform = transform
        self.device = device
    
    def __len__(self) :
        return len(self.image_list)
    
    def __getitem__(self, index) :
        x = cv2.imread(self.image_list[index])
        x = self.transform(x).to(self.device)
        
        y = self.image_labels[index]
        y = torch.LongTensor([y,]).to(self.device) # LongTensor : to intege tensor
        
        return x, y
        
class test_dataset(Dataset) :
    def __init__(self, image_list, transform, device) :
        self.image_list = image_list
        self.transform = transform
        self.image_labels = [int(x[-6:-5]) for x in self.image_list]
        self.device = device
    
    def __len__(self) :
        return len(self.image_list)
    
    def __getitem__(self, index) :
        x = cv2.imread(self.image_list[index])
        x = self.transform(x).to(self.device)
        
        y = self.image_labels[index]
        y = torch.LongTensor([y,]).to(self.device) # LongTensor : to intege tensor
        
        return x, y


In [ ]:
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size = 0.15, 
                                                                  stratify = train_labels, random_state = 42)
len(train_images), len(val_images), len(train_labels), len(val_labels),

In [ ]:
#load train_images
train_data = train_dataset(train_images, train_labels, transform, device)
train_data = DataLoader(train_data, batch_size = 32, shuffle = True) # dataSampler -> shuffle로 대체 가능?

#load val_images
val_data = train_dataset(val_images, val_labels, transform, device)
val_data = DataLoader(val_data, batch_size = 32, shuffle = True)

#load test_images
test_data = test_dataset(test_images, transform, device)
test_data = DataLoader(test_data, batch_size = 32, shuffle = True)

In [ ]:
train_iterator = iter(train_data)

In [ ]:
a1 = next(train_iterator)

In [ ]:
a1[0].shape

In [ ]:
!pip install torchsummaryX

In [ ]:
import torch.nn as nn
from torch.optim import Adam
from torchvision.models import mobilenet_v2
from torchsummaryX import summary

#### MobilNet ####
class mobile(nn.Module) :
    def __init__(self, output) :
        super().__init__()
        self.model = mobilenet_v2(pretrained=True) #use Densenet
        self.model.fc = torch.nn.Linear(1280, output) #change the last FC layer
        
    def forward(self, x) :
        output = self.model(x)
        return output

classes = 6
model = mobile(classes).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

summary(model, torch.rand((1, 3, 224, 224)).float().to(device))

In [ ]:
import torch.nn as nn
from torch.optim import Adam
from torchsummaryX import summary
from torchvision import models

#### ShuffleNet ####
class shuffle(nn.Module) :
    def __init__(self, output) :
        super().__init__()
        self.model = models.shufflenet_v2_x1_0(pretrained=True)
        self.model.fc = torch.nn.Linear(1024, output) #change the last FC layer
        
    def forward(self, x) :
        output = self.model(x)
        return output

classes = 6
model = shuffle(classes).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

summary(model, torch.rand((1, 3, 256, 256)).float().to(device))

In [ ]:
from tqdm.notebook import tqdm

n_epoch = 5
torch.cuda.empty_cache() # ??

## train
model = model.train()

for epoch in range(n_epoch) :
    train_loss = 0
    train_acc = 0
    for x, y in tqdm(train_data) :
        y = y.reshape(-1)
        
        predict = model(x)
        loss = criterion(predict, y)
        
        train_loss += loss / len(train_data)
        correct_prediction = torch.argmax(predict, 1) == y # predict값의 row에서 가장 큰 값 == y면 true?
        correct_prediction = correct_prediction.sum()
        train_acc += correct_prediction
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    train_acc = train_acc / (32 * len(train_data))
    print('Epoch : {}/{},   train_loss : {:.5f},    train_acc : {:.5f}'.format(epoch+1, n_epoch, train_loss, train_acc))
## validation
    val_acc = 0
    val_loss = 0
    with torch.no_grad():
      model.eval()
      for x, y in tqdm(val_data) :
        y = y.reshape(-1)
        
        predict = model(x)
        loss = criterion(predict, y)
        
        val_loss += loss / len(val_data)
        correct_prediction = torch.argmax(predict, 1) == y # predict값의 row에서 가장 큰 값 == y면 true?
        correct_prediction = correct_prediction.sum()
        val_acc += correct_prediction
        
    val_acc = val_acc / (32 * len(val_data))
    print('Epoch : {}/{},   val_loss : {:.5f},    val_acc : {:.5f}'.format(epoch+1, n_epoch, val_loss, val_acc))

    if val_acc > 0.98 : break

In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
import time

start = time.time()
with torch.no_grad() :
    test_loss = 0
    test_acc = 0
    for i, (x, y) in enumerate(test_data) :
        y = y.reshape(-1)
        
        predict = model(x)
        loss = criterion(predict, y)

        test_loss += loss / len(test_data)
        correct_prediction = torch.argmax(predict, 1) == y # [True, Flase, True ...]
        correct_prediction = correct_prediction.sum()
        test_acc += correct_prediction
        
    test_acc = test_acc.item() / (32 * len(test_data))
    
    print("time :", time.time() - start)
    print('loss : {:.5f},    acc : {:.5f}'.format(test_loss, test_acc))


In [ ]:
#torch.save(model, 'ShuffleNet_total_model2.pt') # 전체 모델 저장
#torch.save(model.state_dict(), 'ShuffleNet_model2.pt') # 가중치 저장

In [ ]:
#### 새로운 데이터 test ####
transform_my = transforms.Compose([
    transforms.ToTensor(),  
    transforms.Resize((256, 256)),
    #transforms.Grayscale(num_output_channels=1)
])

In [ ]:
import PIL.Image as pilimg
import matplotlib.pyplot as plt

#image = cv2.imread('../input/fingers/test/00de9808-f8e3-425f-a8b8-204a69256df7_1L.png')
image = cv2.imread('../input/myfinger/img.jpg') # 새로운 손 사진 업로드 후 사용했음.
plt.imshow(image)

In [ ]:
image = transform_my(image).to(device)
image

In [ ]:
image = image.unsqueeze(0)
image.shape

In [ ]:
pred = model(image)
torch.set_printoptions(sci_mode=False) # e+2 이런식 말구 소수로 표현
pred

In [ ]:
torch.argmax(pred, 1)